try https://link.lens.org/ywt8QmB3gAh download

https://github.com/inspirehep/magpie

In [2]:
import pandas as pd
import requests
import time

In [75]:
df=pd.read_json('https://raw.githubusercontent.com/restrepo/cms_pub/main/cms.json')

In [77]:
#df=df.drop('MA_topics',axis='columns')

* https://github.com/inspirehep/rest-api-doc

In order to avoid overwhelming the server, we enforce rate limits per IP address: every IP address is allowed 50 requests, then at most 2 requests per second. If you exceed those limits, you will receive a response with HTTP status code 429 and a x-retry-in header telling you how long to wait before retrying.

In [105]:
i,df.loc[i,"inspire_id"]

(739, '1388363')

In [106]:
r=requests.get(f'https://inspirehep.net/api/literature/{df.loc[i,"inspire_id"]}')

In [4]:
df[739:]

,CMS_id,report,title,journal,date,CTG,category,inspire_id,abstract,keywords,doi
739,92,EXO-12-011,Search for W' decaying to tau lepton and neutr...,PLB 755 (2016) 196,2015-08-18,EXO,Exotica,1388363,The first search for a heavy charged vector bo...,"[p p: scattering, p p: colliding beams, W': se...",10.1016/j.physletb.2016.02.002
740,91,EXO-14-009,Search for a massive resonance decaying into a...,JHEP 02 (2016) 145,2015-06-04,EXO,Exotica,1374621,A search for a massive resonance decaying into...,"[p p: scattering, p p: colliding beams, resona...",10.1007/JHEP02(2016)145
741,90,EXO-12-007,Search for neutral color-octet weak-triplet sc...,JHEP 09 (2015) 201,2015-05-29,EXO,Exotica,1373745,A search for pair production of neutral color-...,"[p p: scattering, p p: colliding beams, jet: b...",10.1007/JHEP09(2015)201
742,89,EXO-14-008,Search for third-generation scalar leptoquarks...,JHEP 07 (2015) 042 [Err: JHEP 11 (2016) 056],2015-03-31,EXO,Exotica,1357201,A search for pair production of third-generati...,"[p p: colliding beams, family: 3, leptoquark: ...",10.1007/JHEP11(2016)056
743,88,EXO-13-007,Search for narrow high-mass resonances in prot...,PLB 748 (2015) 255,2015-02-17,EXO,Exotica,1345160,"A search for a narrow, high-mass resonance dec...","[p p: colliding beams, narrow resonance, reson...",10.1016/j.physletb.2015.07.011
...,...,...,...,...,...,...,...,...,...,...,...
998,5,HIN-10-001,Dependence on pseudorapidity and on centrality...,JHEP 08 (2011) 141,2011-07-24,HIN,Heavy-Ion Physics,,None,,None
999,4,HIN-11-007,Indications of Suppression of Excited $\Upsilo...,PRL 107 (2011) 052302,2011-05-26,HIN,Heavy-Ion Physics,,None,,None
1000,3,HIN-11-001,Long-range and short-range dihadron angular co...,JHEP 07 (2011) 076,2011-05-13,HIN,Heavy-Ion Physics,,None,,None
1001,2,HIN-10-003,Study of Z boson production in PbPb collisions...,PRL 106 (2011) 212301,2011-03-01,HIN,Heavy-Ion Physics,,None,,None


In [ ]:
insprdf=pd.DataFrame()

#See: https://github.com/restrepo/PythonTipsAndTricks/blob/master/pandas/DataFrame.md#insert-list-into-a-cell
df['keywords']=''
df['keywords']=df['keywords'].astype(object)

In [113]:
for i in df.index:
    print(i,end='\r')
    if not df.loc[i,"inspire_id"]:
        continue
    r=requests.get(f'https://inspirehep.net/api/literature/{df.loc[i,"inspire_id"]}')
    try:
        df.loc[i,'abstract']=r.json().get('metadata').get('abstracts')[0].get('value')
    except:
        df.loc[i,'abstract']=''
    try:
        df.at[i,'keywords']=[d.get('value') for d in r.json().get('metadata').get('keywords') 
                              if d.get('schema')=='INSPIRE' ]
    except:
        df.at[i,'keywords']=[]
    try:
        df.loc[i,'doi']=r.json().get('metadata').get('dois')[0].get('value')
    except:
        df.loc[i,'doi']=''

    try:
        insprdf=insprdf.append(r.json(),ignore_index=True).reset_index(drop=True)
    except:
        pass
    time.sleep(0.5)
    #break
insprdf=insprdf.rename({'id':'inspire_id'},axis='columns')    

In [46]:
df.to_csv('cms.csv',index=False)
df.to_json('cms.json',orient='records')

In [119]:
insprdf.columns

Index(['created', 'inspire_id', 'links', 'metadata', 'updated', 'uuid',
       'message', 'status'],
      dtype='object')

In [120]:
insprdf[['created', 'inspire_id', 'links', 'metadata', 'updated', 'uuid']].to_json('inspire.json',orient='records')

In [18]:
#insprdfdf=pd.read_json('inspire.json')

## Check keywords/MA Topics

In [78]:
ln=pd.read_json('MA_topics.json')

In [79]:
ln=ln[ln['doi']!=''].reset_index(drop=True)
ln['doi']=ln['doi'].str.lower()
ln=ln.rename({'doi':'ldoi'},axis='columns')

In [80]:
df['ldoi']=df['doi'].str.lower()

In [81]:
df=df.merge(ln,on='ldoi',how='left')

In [97]:
row='keywords'
#row='MA_topics'

if row=='MA_topics':
    ln2=pd.read_csv('lens-lhc.csv',low_memory=False)
    ln2['Fields of Study']=ln2['Fields of Study'].str.split('; ')
    ln2=ln2[['DOI','Fields of Study']]
    ln2=ln2[ln2['DOI']!=''].reset_index(drop=True)
    ln2=ln2.dropna(subset=['DOI']).reset_index(drop=True)
    ln2['DOI']=ln2['DOI'].str.lower()
    ln2=ln2.rename({'DOI':'ldoi','Fields of Study':'MA_topics'},axis='columns')
    dfm=df[df['MA_topics'].isna()].reset_index(drop=True)
    dfn=df[~df['MA_topics'].isna()].reset_index(drop=True)
    dfm=dfm.drop('MA_topics',axis='columns')
    dfm=dfm.merge(ln2,on='ldoi',how='left')
    df=dfn.append(dfm).reset_index(drop=True)
    df=df.drop('ldoi',axis='columns')
    
kk=df.copy()
    
if row=='keywords':
    kk[row]=kk[row].apply(lambda l: list(set([x.split(':')[0] for x in l])) if isinstance(l,list) else [])
kk=kk[kk[row].apply(lambda l: len(l) if isinstance(l,list) else 0)>0]
kk=kk.reset_index(drop=True)

In [98]:
kk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
EXO    186
HIG    143
SMP    135
TOP    116
HIN     78
SUS     77
B2G     62
BPH     60
FSQ     56
Name: CTG, dtype: int64

In [99]:
kk['CTG'].unique()

array(['FSQ', 'BPH', 'SMP', 'TOP', 'HIG', 'SUS', 'EXO', 'B2G', 'HIN'],
      dtype=object)

In [100]:
unique_row=list(set(kk[row].sum()))

In [101]:
if row=='keywords':
    unique_row=list(set([x for x in unique_row if x.find('-->')==-1])) 

In [102]:
if row=='MA_topics':
    sus=['Minimal Supersymmetric Standard Model',  'Supersymmetry'
       ,'Chargino', 'Gaugino','Gluino','Goldstino','Gravitino','Lightest Supersymmetric Particle','Neutralino','R-parity'
         , 'Split supersymmetry','Stop squark','Superpartner',
     'Supersymmetry breaking'
    ]
    exo=['Dark matter',
     'Dark photon','Extra dimensions', 'Grand Unified Theory', 'Graviton','Hawking radiation','Hidden sector','Higgsino',
     'Kaluza–Klein theory','Leptoquark','Micro black hole', 'Physics beyond the Standard Model','Randall–Sundrum model','Seesaw mechanism',
         'Sterile neutrino','Technicolor','Two-Higgs-doublet model','Universal extra dimension','Unparticle physics','W′ and Z′ bosons','Large extra dimension'
    ]

    hig=['Higgs boson',
     'Higgs field',
     'Higgs mechanism',
     'Higgs sector', 'High mass','Search for the Higgs boson'
        ]

    sm=['Standard Model',
      'Gauge boson', 'W and Z bosons',
    'Cabibbo–Kobayashi–Maskawa matrix',
       ]

    qcd=['Quantum chromodynamics',
     'Gluon','B meson',
     'Bottom quark', 'J/psi meson','Pentaquark','Perturbative QCD','Tetraquark'
     ]
    
    hin=['Heavy ion','Quark–gluon plasma','Relativistic Heavy Ion Collider']

    b2g=['Third generation']

    top=['Top quark']

if row=='keywords':
    sus=[ 'supersymmetry', 'minimal supersymmetric standard model',
     'sneutrino',
     'gravitino',
     'Higgsino',
     'slepton',
     'sbottom',
     'stop',
    'R parity',
      'gaugino',
     'neutralino',
     'gluino',
     'chargino',
     'selectron',
    'bino',
      'sneutrino/tau',
    'smuon',
     'LSP',
     'squark',
     'stau']

    exo=[ 'little Higgs model', 'universal extra dimension', 'technicolor',
          "W'", 'leptoquark','graviton', 'seesaw model', 'unparticle',"Z'",'WIMP','hidden sector',
         'dark matter', 'Kaluza-Klein model','black hole', "top'", 'Randall-Sundrum model',
         'axion-like particles','WIMP nucleon']
    sm=[ 'standard model','CKM matrix']
    qcd=['quantum chromodynamics','gluon', 'tetraquark']

    hig=['Higgs particle'] 

    bph= ['B', 'bottom']
    top=['top'] 
    hin=['heavy ion']

In [103]:
kkk=kk[kk[row].apply(lambda l: list( set( sus ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
SUS    77
EXO    43
HIG    22
B2G     2
TOP     1
Name: CTG, dtype: int64

In [104]:
kkk=kk[kk[row].apply(lambda l: list( set( exo ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
EXO    109
B2G     29
HIG      8
SUS      7
TOP      3
SMP      1
FSQ      1
Name: CTG, dtype: int64

In [105]:
kkk=kk[kk[row].apply(lambda l: list( set( hig ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
HIG    139
B2G     20
EXO     16
SMP      8
TOP      5
SUS      5
FSQ      1
Name: CTG, dtype: int64

In [106]:
kkk=kk[kk[row].apply(lambda l: list( set( top ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
TOP    116
HIG     25
B2G     16
SUS     12
EXO      9
HIN      2
SMP      1
Name: CTG, dtype: int64

In [107]:
kkk=kk[kk[row].apply(lambda l: list( set( hin ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
HIN    38
FSQ     4
Name: CTG, dtype: int64

In [108]:
kkk=kk[kk[row].apply(lambda l: list( set( sm ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
TOP    10
EXO     7
HIG     5
Name: CTG, dtype: int64

In [109]:
kkk=kk[kk[row].apply(lambda l: list( set( qcd ) & set(l) ) ).apply(len)>0]
kkk.groupby('CTG')['CTG'].count().sort_values(ascending=False)

CTG
SMP    60
TOP    28
HIN    17
HIG    14
EXO    11
FSQ     8
BPH     7
B2G     7
SUS     2
Name: CTG, dtype: int64

https://link.lens.org/FoybzNctiBi

In [95]:
lf=kk[row].sum()
lfu=list(set( kk[row].sum() ))
pd.DataFrame(data=zip( lfu, [ lf.count(l) for l in lfu  ] ),columns=['key','count'] ).sort_values('count',ascending=False)[:50]

,key,count
78,Physics,763
258,Particle physics,672
153,Large Hadron Collider,640
272,Nuclear physics,493
105,Standard Model,302
340,Lepton,249
128,Quark,195
141,Muon,195
13,Luminosity (scattering theory),192
43,Boson,184
